코랩과 구글 드라이브 마운트 시킨후 파일 로드

In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


마운트 후 파일 확인

In [0]:
ls -al


total 2102
-rw------- 1 root root 592518 Nov 11 05:39 '김윤환 (1).jpg'
-rw------- 1 root root 276548 Nov 11 05:36 '학사과정 성적증명서_김윤환 (1).pdf'
-rw------- 1 root root    151 Nov 18  2016 '글쓰기2 하이체어 설문.gform'
-rw------- 1 root root  11776 Sep 15 11:42 '[양식3]출장보고서 (1).hwp'
-rw------- 1 root root  11776 Sep 15 11:43 '[양식3]출장보고서.hwp'
-rw------- 1 root root 369305 Jan  4  2012  All-App-Inventor-Projects.zip
drwx------ 2 root root   4096 Nov 11 10:22 'Colab Notebooks'/
-rw------- 1 root root     83 Nov 12 08:37  data-01-test-score.csv
-rw------- 1 root root   3533 Nov 12 12:54  data-04-zoo.csv
-rw------- 1 root root 592518 Nov 11 05:37  김윤환.jpg
drwx------ 2 root root   4096 Nov 12 13:37  ML/
-rw------- 1 root root 276548 Nov 11 05:38 '학사과정 성적증명서_김윤환.pdf'
-rw------- 1 root root   1179 Mar 20  2019  Untitled0.ipynb
-rw------- 1 root root    151 Aug 25  2018 'Untitled document.gdoc'
drwx------ 2 root root   4096 Mar

tensorflow로 파일에서 data 읽어오기

batch를 이용한 multi variable

In [0]:
import tensorflow as tf
filename_queue = tf.train.string_input_producer(['./data-01-test-score.csv'], shuffle=False, name='filename_queue')
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

record_defaults = [[0.],[0.],[0.],[0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)


x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.placeholder(tf.float32, shape=[None, 1])

w = tf.Variable(tf.random_normal([3,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(x, w) + b

cost = tf.reduce_mean(tf.square(hypothesis - y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()

sess.run(tf.global_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess = sess, coord=coord)

for step in range(2001):
  x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
  cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict={x: x_batch, y: y_batch})
  if step % 10 == 0:
    print(step, "Cost: ", cost_val, "\nPrediction: \n", hy_val)

coord.request_stop()
coord.join(threads)

In [0]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)
xy = np.loadtxt('./data-01-test-score.csv', delimiter=',', dtype=np.float32)

x_data = xy[:,0:-1]
y_data = xy[:,[-1]]
print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.placeholder(tf.float32, shape=[None, 1])

w = tf.Variable(tf.random_normal([3,1]), name='weight')

b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(x, w) + b

cost = tf.reduce_mean(tf.square(hypothesis - y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()

sess.run(tf.global_variables_initializer())
for step in range(2001):
  cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict={x: x_data, y: y_data})
  if step % 10 == 0:
    print(step, "Cost: ", cost_val, "\nPrediction: \n", hy_val)

print("Your score will be ", sess.run(hypothesis, feed_dict={x:[[100,70,101]]}))
print("Other scores will be ", sess.run(hypothesis, feed_dict={x:[[60,70,110],[90,100,80]]}))
